In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import time
import sklearn
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)


2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.19.0
pandas 1.0.5
sklearn 0.23.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [ ]:
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

# 注意查看 文件路径
"""kaggle
train_labels_file = '../input/cifar10-object-recognition-in-images-zip-file/trainLabels.csv'
test_csv_file = '../input/cifar10-object-recognition-in-images-zip-file/sampleSubmission.csv'
train_folder = '../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/'
test_folder = '../input/cifar10-object-recognition-in-images-zip-file/train_test/test/test/'

"""
train_labels_file = './cifar10/trainLabels.csv'
test_csv_file = './cifar10/sampleSubmission.csv'
train_folder = './cifar10/train/'
test_folder = './cifar10/test/'

print(os.path.exists(train_folder))
print(os.path.exists(test_folder))
print(os.path.exists(train_labels_file))

In [ ]:
# 读取数据，组装数据
# Parses csv files into (filename(path),label) format
def parse_csv_file(filepath,folder):
    results = []
    with open(filepath,'r') as f:
        lines = f.readlines()[1:]  #不需要header 
    for line in lines:
        image_id,label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder,image_id+'.png')
        results.append((image_full_path,label_str))
    return results

train_labels_info = parse_csv_file(train_labels_file,train_folder)
test_csv_info = parse_csv_file(test_csv_file,test_folder)

import pprint
pprint.pprint(train_labels_info[0:5])
pprint.pprint(test_csv_info[0:5])
print(len(train_labels_info),len(test_csv_info))
    



In [ ]:
# train_df = pd.DataFrame(train_labels_info)
train_df = pd.DataFrame(train_labels_info[0:45000])
valid_df = pd.DataFrame(train_labels_info[45000:])
test_df = pd.DataFrame(test_csv_info)

print(train_df.head())
print(valid_df.head())
print(test_df.head())
# 添加列 别名
train_df.columns = ['filepath','class']
valid_df.columns = ['filepath','class']
test_df.columns = ['filepath','class']

print(train_df.head())
print(valid_df.head())
print(test_df.head())

In [ ]:
# 卷积神经网络无法预处理大小不一致的图片，所以要对读取的图片进行大小变化
# resnet50 图片是  224*224
height = 224
width = 224
channels = 3 
batch_size = 24
num_classes = 10
epochs = 5

# keras.preprocessing.image.ImageDataGenerator
# keras.applications.resnet50.preprocess_input
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
# flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height,width),
    batch_size = batch_size,
    seed = 7,
    shuffle = True,
    class_mode = 'sparse'
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input
)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    calsses = class_names,
    target_size = (height,width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = 'sparse'
)

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num,valid_num)

In [ ]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape,y.shape)
    print(y)


In [ ]:
# keras.applications.ResNet50
resnet50_fine_tune = keras.models.Sequential()
resnet50_fine_tune.add(keras.applications.ResNet50(include_top =False,pooling='avg',weights='imagenet' ))
resnet50_fine_tune.add(keras.layers.Dense(num_classes,activation='softmax'))
resnet50_fine_tune.layers[0].trainable=False
resnet50_fine_tune.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy']) # finetune -> sgd
resnet50_fine_tune.summary()

In [ ]:
# model.fit_generator
history = resnet50_fine_tune.fit_generator(train_generator,steps_per_epoch=train_num//batch_size,epochs=epochs,validation_data=valid_generator,validation_steps=valid_num//batch_size)


In [ ]:
history.history.getkeys()

In [ ]:
def plot_leraning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_leraning_curves(history, 'accuracy', epochs, 0, 1)
plot_leraning_curves(history, 'loss', epochs, 1.0, 2.5)

In [ ]:
# 预测
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height,width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode='sparse'
)

test_num = test_generator.samples
print(test_num)

In [ ]:
# model.predict_generator
test_predict = resnet50_fine_tune.predict_generator(test_generator,workers=10,multiprocessing=False)
print(test_predict.shape)
print(test_predict[0:5])

In [ ]:
test_predict_class_indiecs = np.argmax(test_predict,axis=1) # axis=1行
print(test_predict_class_indiecs[0:5])

In [ ]:
test_predict_class = [class_names[index] for index in test_predict_class_indiecs] #根据index获取class name
print(test_predict_class[0:5])

In [ ]:
# 预测结果写入文件
def generate_submissions(filename,predict_class):
    with open(filename,'w') as f:
        f.write('id,label\n')
        for i in range(len(predict_class)):
            f.write('%d,%s\n'% (i+1,predict_class[i])) # i 从0开始
            
# output_file = "../output/kaggle/working/cifar10-object-recognition-in-images-zip-file/submission.csv"
output_file = "./cifar10/submission.csv"
generate_submissions(output_file,test_predict_class)